In [145]:
# ! pip install openai
# ! pip install google
# ! pip install llama-index-readers-web

In [85]:
import os
from openai import OpenAI
import re
os.environ["OPENAI_API_KEY"] = 'sk-proj-SM3O3571ODLXKnVlqAPkT3BlbkFJPo0oVXq97ZHQhAuGfEMm'
client = OpenAI()

In [32]:
setup_string = """
Your job is to return Google search queries, specifically Google Dorks, to find more
about a subject given a dumped text file of all of the information we currently know
about the subject. Not all of this information should be used in every query.
You can output up to twenty queries, and they should be formatted in a bullet point list
format, where every bullet point contains only the Google Dork search query and nothing
else (without quotes).

The end goal is to create an OSINT (open source intelligence) report about
the given subject. For example, if the user wants to find more about a person,
you would help them find out about their location, age, education, associated
institutions, relevant connections, and so forth.
""" # add osint flowchart

In [33]:
prompt = """
I want to find out more about the person Jai Sharma using Google Dorks. Here are
some things I know about him:
- Went to Monta Vista High School
- Is friends with Milind Maiti
- Might be associated with Berkeley
- Research oriented and possibly interested in Machine Learning, wrote a paper with Christopher Sun
"""

In [36]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": setup_string},
    {"role": "user", "content": prompt}
  ]
)

result = completion.choices[0].message.content
search_list = result.split("\n")
print(result)
print(search_list)

- "Jai Sharma" "Monta Vista High School" site:linkedin.com
- "Jai Sharma" "Milind Maiti" friends site:facebook.com
- "Jai Sharma" Berkeley site:instagram.com
- "Jai Sharma" "Christopher Sun" "Machine Learning" research paper filetype:pdf
- "Jai Sharma" "Machine Learning" site:scholar.google.com
- "Jai Sharma" "Berkeley" "Machine Learning" intitle:resume
- "Jai Sharma" "Research interests" site:researchgate.net
- "Jai Sharma" "Christopher Sun" site:arxiv.org
- "Jai Sharma" "Machine Learning" intitle:publication
- "Jai Sharma" "Google Scholar" "Christopher Sun" "Berkeley"
- "Jai Sharma" "Monta Vista High School" alumni site:linkedin.com
- "Jai Sharma" "Berkeley" "Machine Learning" intitle:thesis
- "Jai Sharma" "Christopher Sun" "Machine Learning" blog
- "Jai Sharma" "Research assistant" "Christopher Sun" site:researchgate.net
- "Jai Sharma" "Machine Learning" intitle:conference presentation
- "Jai Sharma" "Monta Vista High School" graduation year
- "Jai Sharma" "Machine Learning" "Berkel

In [28]:
from googlesearch import search

query = "Geeksforgeeks"

for j in search(query, tld="co.in", num=10, stop=10, pause=2):
    print(j)

https://www.geeksforgeeks.org/
https://in.linkedin.com/company/geeksforgeeks
https://twitter.com/geeksforgeeks?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor
https://twitter.com/geeksforgeeks/status/1807112825149833308?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://twitter.com/geeksforgeeks/status/1807104112074346947?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://twitter.com/geeksforgeeks/status/1807069362639249494?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://twitter.com/geeksforgeeks/status/1807060393480606178?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ
https://www.quora.com/What-is-your-review-of-GeeksforGeeks
https://apps.apple.com/in/app/geeksforgeeks-learn-coding/id1641848816


In [143]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import HTMLNodeParser
urls = ["https://en.wikipedia.org/wiki/Eminem"]
documents = SimpleWebPageReader().load_data(urls)

for i in range(len(documents)):
  documents[i].text = re.sub(r'\\s+', ' ', documents[i].text)
  documents[i].text = re.sub(r' +', ' ', documents[i].text)
  documents[i].text = re.sub(r'\\n', '\n', documents[i].text)
  documents[i].text = re.sub(r'\n +', '\n', documents[i].text)
  documents[i].text = re.sub(r'\n+', '\n', documents[i].text)
  print(documents[i].text)

parser = HTMLNodeParser(
    # tags=["p", "h1"],
    chunk_lines=100,
    chunk_lines_overlap=50,
    max_chars=1500,
    )

nodes = parser.get_nodes_from_documents(documents)
nodes = [node for node in nodes if len(node.text) > 0]
for node in nodes:
  print(node.text)
  print(node.metadata)

Streaming output truncated to the last 5000 lines.
^
van Horn, Teri (June 28, 2001). "Eminem Gets Probation". MTV News. Archived from the original on July 3, 2001. Retrieved January 30, 2008.
{'tag': 'li'}
^
{'tag': 'b'}
^
Bozza 2003, p. 69
{'tag': 'li'}
^
{'tag': 'b'}
^
"Eminem wife's suicide bid". BBC News. July 10, 2000. Archived from the original on January 9, 2009. Retrieved June 14, 2011.
{'tag': 'li'}
^
{'tag': 'b'}
BBC News
{'tag': 'i'}
^
Wiederhorn, Jon (December 10, 2001). "Alleged Bully From Eminem's 'Brain Damage' Files $1 Million Suit". MTV News. Viacom. Archived from the original on February 2, 2012. Retrieved April 3, 2021.
{'tag': 'li'}
^
{'tag': 'b'}
MTV News
{'tag': 'i'}
^
Dansby, Andrew (October 20, 2003). "Judge Drops Eminem Rap". Rolling Stone. Archived from the original on July 6, 2015. Retrieved August 12, 2015.
{'tag': 'li'}
^
{'tag': 'b'}
Rolling Stone
{'tag': 'i'}
^
"Eminem sued by jazz star". BBC News. March 31, 2002. Archived from the original on August 1, 2

In [142]:
len(nodes)

2374